In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from functools import reduce

In [2]:
all_data_frames = []

In [3]:
populations = pd.read_csv("data/economic/Population/data.csv")
print("countries count:", len(populations["Country or Area"].unique()))

replacement_df = pd.DataFrame(columns = ["Country", "Population (thousands)"])
for country in populations["Country or Area"].unique():
    sub = populations[populations["Country or Area"] == country]
    years = sub["Year(s)"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent rate
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["Year(s)"] == year_to_use].values[0, -2]]

populations = replacement_df

all_data_frames.append(populations)
populations

countries count: 194


,Country,Population (thousands)
0,Afghanistan,29825
1,Albania,3162
2,Algeria,38482
3,Andorra,78
4,Angola,20821
...,...,...
189,Venezuela (Bolivarian Republic of),29955
190,Viet Nam,90796
191,Yemen,23852
192,Zambia,14075


In [4]:
# USE THIS FOR WORLD BANK DATA

compulsory_attendence_duration = pd.read_csv("data/education/attendence/compulsory_duration.csv", skiprows = 3)
compulsory_attendence_duration = compulsory_attendence_duration.drop(["Country Code", "Indicator Name", "Indicator Code"], axis = "columns")

print("countries count:", len(compulsory_attendence_duration["Country Name"].unique()))

replacement_df = pd.DataFrame(columns = ["Country", "Compulsory attendence amount"])
for country in compulsory_attendence_duration["Country Name"].unique():
    sub = compulsory_attendence_duration[compulsory_attendence_duration["Country Name"] == country]

    col = len(sub.values[0]) - 2
    while col > 1 and str(sub.values[0, col]) == "nan":
        col = col - 1
        
    replacement_df.loc[len(replacement_df.index)] = [country, sub.values[0, col]]

compulsory_attendence_duration = replacement_df

all_data_frames.append(compulsory_attendence_duration)

compulsory_attendence_duration

countries count: 266


,Country,Compulsory attendence amount
0,Aruba,13.0
1,Africa Eastern and Southern,7.0
2,Afghanistan,9.0
3,Africa Western and Central,9.0
4,Angola,6.0
...,...,...
261,Kosovo,NaN
262,"Yemen, Rep.",9.0
263,South Africa,9.0
264,Zambia,7.0


In [5]:
early_education_attendence_rates = pd.read_csv("data/education/attendence/early_education_rates.csv")
early_education_attendence_rates.drop(early_education_attendence_rates.tail(5).index, inplace = True)

print("countries count:", len(early_education_attendence_rates["Country or Area"].unique()))
# display(early_education_attendence_rates["Subgroup"].value_counts())

early_education_attendence_rates = early_education_attendence_rates[early_education_attendence_rates["Subgroup"] == "Total"]
early_education_attendence_rates = early_education_attendence_rates.iloc[:, [0, -2]].rename(columns = {"Country or Area": "Country", "Value": "Early education attendance rate"})
all_data_frames.append(early_education_attendence_rates)
early_education_attendence_rates

countries count: 58


,Country,Early education attendance rate
0,Afghanistan,1.0
5,Albania,40.0
10,Bangladesh,15.0
15,Belarus,86.0
18,Belize,32.0
23,Bhutan,10.0
28,Bosnia and Herzegovina,6.0
33,Botswana,18.0
34,Burkina Faso,2.0
39,Burundi,5.0


In [6]:
# USE THIS FOR UN DATA

enrollment_primary = pd.read_csv("data/education/enrollment/primary.csv")
enrollment_primary.drop(enrollment_primary.tail(9).index, inplace = True)
print("countries count:", len(enrollment_primary["Country or Area"].unique()))
# print(enrollment_primary["Year"].value_counts())

replacement_df = pd.DataFrame(columns = ["Country", "Primary enrollment"])
for country in enrollment_primary["Country or Area"].unique():
    sub = enrollment_primary[enrollment_primary["Country or Area"] == country]
    years = sub["Year"].unique().tolist()
    years.sort(reverse=True)
    y = 0
    value_counts = sub["Year"].value_counts()
    while not value_counts[years[y]] == 2:
        y = y + 1
    
    # Grab male and female count for most recent year with both
    #                                ^^^^^^^^^^^^^^^^^^^^^^^^^^
    #                                calculated above
    sub2 = sub[sub["Year"] == years[y]]
    male_count = sub2[sub2["Subgroup"] == "Male"].values[0, -2]
    female_count = sub2[sub2["Subgroup"] == "Female"].values[0, -2]
    total_enrollment = int(male_count) + int(female_count)

    replacement_df.loc[len(replacement_df.index)] = [country, total_enrollment]

enrollment_primary = replacement_df

all_data_frames.append(enrollment_primary)
enrollment_primary

countries count: 201


,Country,Primary enrollment
0,Afghanistan,4318819
1,Albania,250487
2,Algeria,4361744
3,Andorra,4085
4,Angola,1057188
...,...,...
196,Venezuela,3449290
197,Vietnam,7773484
198,Yemen,3219554
199,Zambia,2565419


In [7]:
enrollment_secondary = pd.read_csv("data/education/enrollment/secondary.csv")
enrollment_secondary.drop(enrollment_secondary.tail(6).index, inplace = True)
print("countries count:", len(enrollment_secondary["Country or Area"].unique()))

replacement_df = pd.DataFrame(columns = ["Country", "Secondary enrollment"])
for country in enrollment_secondary["Country or Area"].unique():
    sub = enrollment_secondary[enrollment_secondary["Country or Area"] == country]
    years = sub["Year"].unique().tolist()
    years.sort(reverse=True)
    y = 0
    value_counts = sub["Year"].value_counts()
    while not value_counts[years[y]] == 2:
        y = y + 1
    
    # Grab male and female count for most recent year with both
    #                                ^^^^^^^^^^^^^^^^^^^^^^^^^^
    #                                calculated above
    sub2 = sub[sub["Year"] == years[y]]
    male_count = sub2[sub2["Subgroup"] == "Male"].values[0, -2]
    female_count = sub2[sub2["Subgroup"] == "Female"].values[0, -2]
    total_enrollment = int(male_count) + int(female_count)

    replacement_df.loc[len(replacement_df.index)] = [country, total_enrollment]

enrollment_secondary = replacement_df

all_data_frames.append(enrollment_secondary)
enrollment_secondary

countries count: 200


,Country,Secondary enrollment
0,Afghanistan,651453
1,Albania,397056
2,Algeria,3755821
3,Andorra,3737
4,Angola,413695
...,...,...
195,Venezuela,2028388
196,Vietnam,9939319
197,Yemen,1455216
198,Zambia,408971


In [8]:
enrollment_tertiary = pd.read_csv("data/education/enrollment/tertiary.csv")
enrollment_tertiary.drop(enrollment_tertiary.tail(6).index, inplace = True)
print("countries count:", len(enrollment_tertiary["Country or Area"].unique()))

enrollment_tertiary.drop(enrollment_tertiary[enrollment_tertiary["Value"] == "…" ].index , inplace = True)
enrollment_tertiary["Value"].value_counts()

replacement_df = pd.DataFrame(columns = ["Country", "Tertiary enrollment"])
for country in enrollment_tertiary["Country or Area"].unique():
    sub = enrollment_tertiary[enrollment_tertiary["Country or Area"] == country]
    years = sub["Year"].unique().tolist()
    years.sort(reverse=True)
    y = 0
    value_counts = sub["Year"].value_counts()
    while not value_counts[years[y]] == 2:
        y = y + 1
    
    # Grab male and female count for most recent year with both
    #                                ^^^^^^^^^^^^^^^^^^^^^^^^^^
    #                                calculated above
    sub2 = sub[sub["Year"] == years[y]]
    male_count = sub2[sub2["Subgroup"] == "Male"].values[0, -2]
    female_count = sub2[sub2["Subgroup"] == "Female"].values[0, -2]
    total_enrollment = int(male_count) + int(female_count)

    replacement_df.loc[len(replacement_df.index)] = [country, total_enrollment]

enrollment_tertiary = replacement_df

all_data_frames.append(enrollment_tertiary)
enrollment_tertiary

countries count: 191


,Country,Tertiary enrollment
0,Afghanistan,27648
1,Albania,53014
2,Algeria,755463
3,Andorra,342
4,Angola,12566
...,...,...
167,Venezuela,983217
168,Vietnam,1354543
169,Yemen,201043
170,Zambia,24553


In [9]:
literacy_rates = pd.read_csv("data/education/literacy/15+.csv")
literacy_rates.drop(literacy_rates.tail(2).index, inplace = True)
print("countries count:", len(literacy_rates["Country or Area"].unique()))
literacy_rates = literacy_rates.iloc[ :, [ 0, 1, 2, 5 ] ]

replacement_df = pd.DataFrame(columns = ["Country", "Male literacy rate", "Female literacy rate"])
for country in literacy_rates["Country or Area"].unique():
    sub = literacy_rates[literacy_rates["Country or Area"] == country]
    years = sub["Year"].unique().tolist()
    years.sort(reverse=True)
    y = 0
    value_counts = sub["Year"].value_counts()
    while not value_counts[years[y]] == 2:
        y = y + 1
    
    # Grab male and female count for most recent year with both
    #                                ^^^^^^^^^^^^^^^^^^^^^^^^^^
    #                                calculated above
    sub2 = sub[sub["Year"] == years[y]]
    male_rate = sub2[sub2["Subgroup"] == "Male 15+ yr"].values[0, -1]
    female_rate = sub2[sub2["Subgroup"] == "Female 15+ yr"].values[0, -1]

    replacement_df.loc[len(replacement_df.index)] = [country, male_rate, female_rate]

literacy_rates = replacement_df

all_data_frames.append(literacy_rates)
literacy_rates

countries count: 135


,Country,Male literacy rate,Female literacy rate
0,Afghanistan,43.1,12.6
1,Albania,99.2,98.3
2,Algeria,79.6,60.1
3,Angola,82.9,54.2
4,Argentina,97.2,97.2
...,...,...,...
130,Venezuela,93.3,92.7
131,Vietnam,93.9,86.9
132,Yemen,73.1,34.7
133,Zambia,76.3,59.8


In [10]:
primary_completion_rates = pd.read_csv("data/education/school_life_expectancy/drop_out_rates/primary_completion.csv", skiprows = 3)
primary_completion_rates = primary_completion_rates.drop(["Country Code", "Indicator Code", "Indicator Name"], "columns")

print("countries count:", len(primary_completion_rates["Country Name"].unique()))

replacement_df = pd.DataFrame(columns = ["Country", "Primary completion rate"])
for country in primary_completion_rates["Country Name"].unique():
    sub = primary_completion_rates[primary_completion_rates["Country Name"] == country]

    col = len(sub.values[0]) - 2
    while col > 1 and str(sub.values[0, col]) == "nan":
        col = col - 1
        
    replacement_df.loc[len(replacement_df.index)] = [country, sub.values[0, col]]

primary_completion_rates = replacement_df

all_data_frames.append(primary_completion_rates)

primary_completion_rates


countries count: 266


,Country,Primary completion rate
0,Aruba,101.180557
1,Africa Eastern and Southern,NaN
2,Afghanistan,84.330589
3,Africa Western and Central,NaN
4,Angola,46.187149
...,...,...
261,Kosovo,NaN
262,"Yemen, Rep.",72.300392
263,South Africa,91.999489
264,Zambia,79.973877


In [11]:
primary_to_secondary_transition_rates = pd.read_csv("data/education/school_life_expectancy/drop_out_rates/primary_to_secondary_transition.csv")
print("countries count:", len(primary_to_secondary_transition_rates["Reference Area"].unique()))
primary_to_secondary_transition_rates = primary_to_secondary_transition_rates[primary_to_secondary_transition_rates["Sex"] == "All genders"]

replacement_df = pd.DataFrame(columns = ["Country", "Primary to secondary transition rate"])
for country in primary_to_secondary_transition_rates["Reference Area"].unique():
    sub = primary_to_secondary_transition_rates[primary_to_secondary_transition_rates["Reference Area"] == country]
    years = sub["Time Period"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent rate
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["Time Period"] == year_to_use].values[0, -1]]

primary_to_secondary_transition_rates = replacement_df

all_data_frames.append(primary_to_secondary_transition_rates)
primary_to_secondary_transition_rates

countries count: 193


,Country,Primary to secondary transition rate
0,Afghanistan,56.14616
1,Albania,99.05388
2,Algeria,94.39702
3,Angola,44.15628
4,Anguilla,96.83601
...,...,...
188,Venezuela (Bolivarian Republic of),98.68418
189,Viet Nam,87.05014
190,Yemen,83.30139
191,Zambia,59.15526


In [12]:
primary_drop_out_rates = pd.read_csv("data/education/school_life_expectancy/drop_out_rates/primary.csv")
print("countries count:", len(primary_drop_out_rates["Reference Area"].unique()))
primary_drop_out_rates = primary_drop_out_rates[primary_drop_out_rates["Sex"] == "All genders"]

replacement_df = pd.DataFrame(columns = ["Country", "Primary drop out rate"])
for country in primary_drop_out_rates["Reference Area"].unique():
    sub = primary_drop_out_rates[primary_drop_out_rates["Reference Area"] == country]
    years = sub["Time Period"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent rate
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["Time Period"] == year_to_use].values[0, -1]]

primary_drop_out_rates = replacement_df

all_data_frames.append(primary_drop_out_rates)
primary_drop_out_rates

countries count: 187


,Country,Primary drop out rate
0,Afghanistan,12.21400
1,Albania,1.28747
2,Algeria,6.58700
3,Andorra,29.34941
4,Angola,68.12715
...,...,...
182,Venezuela (Bolivarian Republic of),12.85023
183,Viet Nam,10.40116
184,Yemen,30.52896
185,Zambia,44.51099


In [13]:
lower_secondary_repeaters = pd.read_csv("data/education/school_life_expectancy/repeaters/lower_secondary_counts.csv")

print("countries count:", len(lower_secondary_repeaters["Reference Area"].unique()))
lower_secondary_repeaters = lower_secondary_repeaters[lower_secondary_repeaters["Sex"] == "All genders"]

replacement_df = pd.DataFrame(columns = ["Country", "Lower secondary repeaters count"])
for country in lower_secondary_repeaters["Reference Area"].unique():
    sub = lower_secondary_repeaters[lower_secondary_repeaters["Reference Area"] == country]
    years = sub["Time Period"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent count
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["Time Period"] == year_to_use].values[0, -1]]

lower_secondary_repeaters = replacement_df

all_data_frames.append(lower_secondary_repeaters)
lower_secondary_repeaters

countries count: 207


,Country,Lower secondary repeaters count
0,Afghanistan,19907.0
1,Albania,1998.0
2,Algeria,536757.0
3,Andorra,183.0
4,Angola,66986.0
...,...,...
202,Venezuela (Bolivarian Republic of),80537.0
203,Viet Nam,54514.0
204,Yemen,79255.0
205,Zambia,48206.0


In [14]:
primary_repeaters = pd.read_csv("data/education/school_life_expectancy/repeaters/primary_counts.csv")

print("countries count:", len(primary_repeaters["Reference Area"].unique()))
primary_repeaters = primary_repeaters[primary_repeaters["Sex"] == "All genders"]

replacement_df = pd.DataFrame(columns = ["Country", "Primary repeaters count"])
for country in primary_repeaters["Reference Area"].unique():
    sub = primary_repeaters[primary_repeaters["Reference Area"] == country]
    years = sub["Time Period"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent count
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["Time Period"] == year_to_use].values[0, -1]]

primary_repeaters = replacement_df

all_data_frames.append(primary_repeaters)
primary_repeaters

countries count: 212


,Country,Primary repeaters count
0,Afghanistan,70786.0
1,Albania,1435.0
2,Algeria,260056.0
3,Andorra,58.0
4,Angola,517629.0
...,...,...
207,Venezuela (Bolivarian Republic of),104570.0
208,Viet Nam,73925.0
209,Yemen,326264.0
210,Zambia,169961.0


In [15]:
primary_repeat_percentages = pd.read_csv("data/education/school_life_expectancy/repeaters/primary_percentages.csv")

print("countries count:", len(primary_repeat_percentages["Reference Area"].unique()))
primary_repeat_percentages = primary_repeat_percentages[primary_repeat_percentages["Sex"] == "All genders"]

replacement_df = pd.DataFrame(columns = ["Country", "Primary repeaters rate"])
for country in primary_repeat_percentages["Reference Area"].unique():
    sub = primary_repeat_percentages[primary_repeat_percentages["Reference Area"] == country]
    years = sub["Time Period"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent rate
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["Time Period"] == year_to_use].values[0, -1]]

primary_repeat_percentages = replacement_df

all_data_frames.append(primary_repeat_percentages)
primary_repeat_percentages

countries count: 203


,Country,Primary repeaters rate
0,Afghanistan,8.99976
1,Albania,0.73319
2,Algeria,6.90663
3,Andorra,1.42681
4,Angola,10.29738
...,...,...
198,Venezuela (Bolivarian Republic of),3.00046
199,Viet Nam,0.99430
200,Yemen,8.42028
201,Zambia,5.52690


In [16]:
school_life_expectancy = pd.read_csv("data/education/school_life_expectancy/expectancy.csv")

print("countries count:", len(school_life_expectancy["Reference Area"].unique()))
school_life_expectancy = school_life_expectancy[school_life_expectancy["Sex"] == "All genders"]

replacement_df = pd.DataFrame(columns = ["Country", "School life expectancy"])
for country in school_life_expectancy["Reference Area"].unique():
    sub = school_life_expectancy[school_life_expectancy["Reference Area"] == country]
    years = sub["Time Period"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent rate
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["Time Period"] == year_to_use].values[0, -1]]

school_life_expectancy = replacement_df

all_data_frames.append(school_life_expectancy)
school_life_expectancy

countries count: 203


,Country,School life expectancy
0,Afghanistan,6.71741
1,Albania,5.54973
2,Algeria,5.95081
3,Andorra,5.71878
4,Angola,8.11996
...,...,...
198,Venezuela (Bolivarian Republic of),6.05845
199,Viet Nam,5.46856
200,Yemen,5.90685
201,Zambia,7.46458


In [17]:
teacher_student_ratios_lower_secondary = pd.read_csv("data/education/teacher_student_ratio/lower_secondary.csv")

print("countries count:", len(teacher_student_ratios_lower_secondary["Reference Area"].unique()))

replacement_df = pd.DataFrame(columns = ["Country", "Lower secondary teacher-student ratio"])
for country in teacher_student_ratios_lower_secondary["Reference Area"].unique():
    sub = teacher_student_ratios_lower_secondary[teacher_student_ratios_lower_secondary["Reference Area"] == country]
    years = sub["Time Period"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent rate
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["Time Period"] == year_to_use].values[0, -1]]

teacher_student_ratios_lower_secondary = replacement_df

all_data_frames.append(teacher_student_ratios_lower_secondary)
teacher_student_ratios_lower_secondary

countries count: 175


,Country,Lower secondary teacher-student ratio
0,Afghanistan,45.04142
1,Albania,12.33030
2,Algeria,17.80495
3,Andorra,7.25659
4,Angola,33.27119
...,...,...
170,Uruguay,10.13620
171,Vanuatu,17.77791
172,Viet Nam,15.62477
173,Yemen,11.09061


In [18]:
teacher_student_ratios_primary = pd.read_csv("data/education/teacher_student_ratio/primary.csv")

print("countries count:", len(teacher_student_ratios_primary["Reference Area"].unique()))

replacement_df = pd.DataFrame(columns = ["Country", "Primary teacher-student ratio"])
for country in teacher_student_ratios_primary["Reference Area"].unique():
    sub = teacher_student_ratios_primary[teacher_student_ratios_primary["Reference Area"] == country]
    years = sub["Time Period"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent rate
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["Time Period"] == year_to_use].values[0, -1]]

teacher_student_ratios_primary = replacement_df

all_data_frames.append(teacher_student_ratios_primary)
teacher_student_ratios_primary

countries count: 206


,Country,Primary teacher-student ratio
0,Afghanistan,45.71171
1,Albania,18.92294
2,Algeria,23.66406
3,Andorra,9.51991
4,Angola,42.54306
...,...,...
201,Venezuela (Bolivarian Republic of),25.68126
202,Viet Nam,19.21383
203,Yemen,30.25279
204,Zambia,47.94526


In [19]:
teacher_student_ratios_upper_secondary = pd.read_csv("data/education/teacher_student_ratio/upper_secondary.csv")

print("countries count:", len(teacher_student_ratios_upper_secondary["Reference Area"].unique()))

replacement_df = pd.DataFrame(columns = ["Country", "Upper secondary teacher-student ratio"])
for country in teacher_student_ratios_upper_secondary["Reference Area"].unique():
    sub = teacher_student_ratios_upper_secondary[teacher_student_ratios_upper_secondary["Reference Area"] == country]
    years = sub["Time Period"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent rate
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["Time Period"] == year_to_use].values[0, -1]]

teacher_student_ratios_upper_secondary = replacement_df

all_data_frames.append(teacher_student_ratios_upper_secondary)
teacher_student_ratios_upper_secondary

countries count: 171


,Country,Upper secondary teacher-student ratio
0,Afghanistan,30.74908
1,Albania,17.65478
2,Algeria,16.15822
3,Andorra,13.98529
4,Angola,20.50581
...,...,...
166,United States of America,14.77404
167,Uruguay,13.78374
168,Vanuatu,14.82099
169,Viet Nam,27.60630


In [20]:
education_spending_perecentage_GDP = pd.read_csv("data/education_spending/percentage_GDP.csv", skiprows = 3)
education_spending_perecentage_GDP = education_spending_perecentage_GDP.drop(["Country Code", "Indicator Code", "Indicator Name"], "columns")

print("countries count:", len(education_spending_perecentage_GDP["Country Name"].unique()))

replacement_df = pd.DataFrame(columns = ["Country", "Education spending (percentage of GDP)"])
for country in education_spending_perecentage_GDP["Country Name"].unique():
    sub = education_spending_perecentage_GDP[education_spending_perecentage_GDP["Country Name"] == country]

    col = len(sub.values[0]) - 2
    while col > 1 and str(sub.values[0, col]) == "nan":
        col = col - 1
        
    replacement_df.loc[len(replacement_df.index)] = [country, sub.values[0, col]]

education_spending_perecentage_GDP = replacement_df

all_data_frames.append(education_spending_perecentage_GDP)

education_spending_perecentage_GDP

countries count: 266


,Country,Education spending (percentage of GDP)
0,Aruba,5.49135
1,Africa Eastern and Southern,4.71930
2,Afghanistan,3.21378
3,Africa Western and Central,3.03285
4,Angola,1.82118
...,...,...
261,Kosovo,NaN
262,"Yemen, Rep.",5.15143
263,South Africa,6.83613
264,Zambia,4.46518


In [21]:
unemployment_rates = pd.read_csv("data/economic/UnemploymentRate/UNE_2EAP_SEX_AGE_RT_A-full-2021-11-12.csv")
unemployment_rates = unemployment_rates[unemployment_rates["sex.label"] == "Sex: Total"]

print("countries count:", len(unemployment_rates["ref_area.label"].unique()))

replacement_df = pd.DataFrame(columns = ["Country", "Unemployment rate"])
for country in unemployment_rates["ref_area.label"].unique():
    sub = unemployment_rates[unemployment_rates["ref_area.label"] == country]
    years = sub["time"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent rate
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["time"] == year_to_use].values[0, -2]]

unemployment_rates = replacement_df

all_data_frames.append(unemployment_rates)

unemployment_rates

countries count: 283


,Country,Unemployment rate
0,Afghanistan,11.73
1,Angola,7.70
2,Albania,11.70
3,United Arab Emirates,5.00
4,Argentina,11.67
...,...,...
278,Central and Western Asia: Low income,7.46
279,Yemen,13.42
280,South Africa,28.74
281,Zambia,12.17


In [22]:
ChildLabor = pd.read_csv("data/economic/ChildLabor/UNChildLabor.csv",skiprows=[i for i in range(337,345)])
ChildLabor = ChildLabor.drop(columns=["Year", "Source","Value Footnotes"])
ChildLabor_filtered = ChildLabor[ChildLabor["Subgroup"]=="Total 5-14 yr"]
ChildLabor_filtered = ChildLabor_filtered.rename(columns={"Country or Area": "Country", "Value": "Child labor percentage"})
ChildLabor_filtered = ChildLabor_filtered.drop(columns=["Subgroup", "Unit"])
all_data_frames.append(ChildLabor_filtered)
ChildLabor_filtered

,Country,Child labor percentage
2,Afghanistan,10
5,Albania,12
8,Algeria,5
11,Angola,24
14,Argentina,7
...,...,...
322,Viet Nam,7
325,Western and Central Africa,27
328,World,15
331,Yemen,23


In [23]:
AvgIncome = pd.read_csv("data/economic/AvgIncome/UNdata_Export_20211112_234256215.csv",skiprows=[2892,2893,2894])
AvgIncome = AvgIncome[["Country or Area","Year","Value","Currency"]]


df = pd.concat([
    sub[sub.Year == sub.Year.max()][['Country or Area','Year','Value','Currency']]
    for country,sub in AvgIncome.groupby("Country or Area")
]).drop_duplicates(subset=['Country or Area']).reset_index(drop=True)
  
df = df.drop(columns=['Year'])
df = df.rename(columns={"Country or Area": "Country", "Value": "Average income", "Currency": "Average income currency"})
all_data_frames.append(df)
df

,Country,Average income,Average income currency
0,Armenia,1.910260e+11,dram
1,Aruba,9.300000e+07,Aruban florin
2,Austria,1.034430e+10,Euro
3,Bahrain,2.867010e+08,Bahrain dinar
4,Belarus,5.052280e+08,Belarussian rouble (re-denom. 1:10000)
...,...,...,...
85,United States,1.239681e+12,US dollar
86,Uruguay,5.017741e+10,Uruguayan peso
87,Uzbekistan,6.364698e+12,Uzbek sum
88,Vanuatu,1.949900e+10,vatu


In [24]:
AvgIncome_Median = pd.read_csv("data/economic/AvgIncome/Median/country-median-data-2011-PPP-Diofasi-Birdsall.csv")
AvgIncome_Median = AvgIncome_Median[["Country","Median income/consumption"]]
AvgIncome_Median

,Country,Median income/consumption
0,Albania,6.50
1,Angola,2.90
2,Armenia,5.00
3,Australia,44.00
4,Austria,46.80
...,...,...
144,Vanuatu,3.65
145,Venezuela,8.40
146,Vietnam,6.15
147,West Bank and Gaza,8.50


In [25]:
avg_income_gni = pd.read_csv("data/economic/AvgIncome/GNI/API_NY.GNP.PCAP.CD_DS2_en_csv_v2_3159868.csv", skiprows = 3)
avg_income_gni = avg_income_gni.drop(["Country Code", "Indicator Code", "Indicator Name"], "columns")

print("countries count:", len(avg_income_gni["Country Name"].unique()))

replacement_df = pd.DataFrame(columns = ["Country", "Average Income GNI"])
for country in avg_income_gni["Country Name"].unique():
    sub = avg_income_gni[avg_income_gni["Country Name"] == country]

    col = len(sub.values[0]) - 2
    while col > 1 and str(sub.values[0, col]) == "nan":
        col = col - 1
        
    replacement_df.loc[len(replacement_df.index)] = [country, sub.values[0, col]]

avg_income_gni = replacement_df

all_data_frames.append(avg_income_gni)

avg_income_gni

countries count: 266


,Country,Average Income GNI
0,Aruba,27120.000000
1,Africa Eastern and Southern,1368.425717
2,Afghanistan,500.000000
3,Africa Western and Central,1646.334568
4,Angola,2230.000000
...,...,...
261,Kosovo,4440.000000
262,"Yemen, Rep.",940.000000
263,South Africa,5410.000000
264,Zambia,1190.000000


In [26]:
current_gdp_usd = pd.read_csv("data/economic/GDP/gdp_current.csv", skiprows = 3)
current_gdp_usd = current_gdp_usd.drop(["Country Code", "Indicator Name", "Indicator Code"], axis = "columns")

print("countries count:", len(current_gdp_usd["Country Name"].unique()))

replacement_df = pd.DataFrame(columns = ["Country", "GDP (USD)"])
for country in current_gdp_usd["Country Name"].unique():
    sub = current_gdp_usd[current_gdp_usd["Country Name"] == country]

    col = len(sub.values[0]) - 2
    while col > 1 and str(sub.values[0, col]) == "nan":
        col = col - 1
        
    replacement_df.loc[len(replacement_df.index)] = [country, sub.values[0, col]]

current_gdp_usd = replacement_df

all_data_frames.append(current_gdp_usd)

current_gdp_usd

countries count: 266


,Country,GDP (USD)
0,Aruba,3.202189e+09
1,Africa Eastern and Southern,9.008286e+11
2,Afghanistan,1.980707e+10
3,Africa Western and Central,7.865850e+11
4,Angola,6.230691e+10
...,...,...
261,Kosovo,7.611402e+09
262,"Yemen, Rep.",2.348627e+10
263,South Africa,3.019236e+11
264,Zambia,1.932005e+10


In [27]:
labor_force = pd.read_csv("data/economic/LaborStats/labor_statistics_un.csv")
print("countries count:", len(labor_force["Country or Area"].unique()))
temp1 = labor_force[labor_force["Description"] == "LABOUR FORCE (IN THOUSANDS)"]
temp2 = labor_force[labor_force["Description"] == "LABOR FORCE (IN THOUSANDS)"]
temp3 = labor_force[labor_force["Description"] == "LABOUR FORCE"]
temp4 = labor_force[labor_force["Description"] == "LABOR FORCE"]
temp5 = labor_force[labor_force["Description"] == "LABOURFORCE(INTHOUSANDS)"]

frames = [temp1,temp2, temp3, temp4, temp5]
labor_force = pd.concat(frames)

replacement_df = pd.DataFrame(columns = ["Country", "Labor force count"])
for country in labor_force["Country or Area"].unique():
    sub = labor_force[labor_force["Country or Area"] == country]
    years = sub["Year"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]

    # Grab most recent rate
    replacement_df.loc[len(replacement_df.index)] = [country.lower().title(), sub[sub["Year"] == year_to_use].values[0, -1]]
    replacement_df

labor_force = replacement_df

all_data_frames.append(labor_force)
labor_force

countries count: 129


,Country,Labor force count
0,Algeria,10500000.0
1,Croatia,1766440.0
2,"Macedonia, Fyr",928774.0
3,Malaysia,10880300.0
4,Albania,1102890.0
...,...,...
96,Uruguay,1580450.0
97,"Venezuela, Rep. Bol.",12379700.0
98,Zimbabwe,4963260.0
99,Bulgaria,3467200.0


In [28]:
three_year_average__unnourished = pd.read_csv("data/economic/FoodDeficits/3YearAvg.csv")
print("countries count:", len(three_year_average__unnourished["Country or Area"].unique()))

three_year_average__unnourished = three_year_average__unnourished[three_year_average__unnourished["Element"] == "Value"]

replacement_df = pd.DataFrame(columns = ["Country", "Three year average unnourished count"])
for country in three_year_average__unnourished["Country or Area"].unique():
    sub = three_year_average__unnourished[three_year_average__unnourished["Country or Area"] == country]
    years = sub["Year"].unique().tolist()
    years.sort(reverse=True)
    year_to_use = years[0]
    # Grab most recent rate
    replacement_df.loc[len(replacement_df.index)] = [country, sub[sub["Year"] == year_to_use].values[0,-2]]

three_year_average__unnourished = replacement_df

all_data_frames.append(three_year_average__unnourished)
three_year_average__unnourished

countries count: 137


,Country,Three year average unnourished count
0,Afghanistan,11.1
1,Africa,239.6
2,Albania,0.1
3,Algeria,1.2
4,Angola,5.7
...,...,...
130,Viet Nam,6.1
131,Western Africa,56.1
132,Western Asia,30.2
133,Western Asia and Northern Africa,45.6


In [29]:
replace_names_dict = {
    "Bolivia (Plurinational State of)": "Bolivia",
    "Bahamas, The": "Bahamas",
    "Côte d'Ivoire": "Cote d'Ivoire",
    "Côte d’Ivoire": "Cote d'Ivoire",
    "Egypt, Arab Rep.": "Egypt",
    "Congo, Dem. Rep.": "Congo",
    "Curaçao": "Curacao",
    "Gambia The": "Gambia",
    "Gambia, The": "Gambia",
    "Hong Kong SAR": "Hong Kong SAR, China",
    "Hong Kong SAR, China": "Hong Kong SAR, China",
    "Hong Kong Special Administrative Region of China": "Hong Kong SAR, China",
    "Hong Kong, China": "Hong Kong SAR, China",
    "China,P.R.:Hong Kong": "Hong Kong SAR, China",
    "Iran (Islamic Republic of)": "Iran",
    "Iran, Islamic Rep.": "Iran",
    "Iran, Islamic Republic of": "Iran",
    "Korea, Republic Of": "Democratic People's Republic of Korea",
    "Korea Rep": "Democratic People's Republic of Korea",
    "Korea, Rep.": "Democratic People's Republic of Korea",
    "Korea, Republic of": "Democratic People's Republic of Korea",
    "Macao SAR, China": "Macao (Macau) SAR, China",
    "Macao Special Administrative Region of China": "Macao (Macau) SAR, China",
    "Macau SAR": "Macao (Macau) SAR, China",
    "Macau, China": "Macao (Macau) SAR, China",
    "China,P.R.:Macao": "Macao (Macau) SAR, China",
    "Micronesia (Federated States of)": "Micronesia",
    "Micronesia, Fed. Sts.": "Micronesia",
    "Micronesia, Federated States of": "Micronesia",
    "Moldova, Republic of": "Moldova",
    "Republic of Moldova": "Moldova",
    "St Vincent and the Grenadines": "St. Vincent and the Grenadines",
    "Tanzania, United Republic of": "Tanzania",
    "United Republic of Tanzania": "Tanzania",
    "Timor Leste": "Timor-Leste",
    "The former Yugoslav Republic of Macedonia": "Macedonia",
    "United States": "United States of America",
    "Viet Nam": "Vietnam",
    "Venezuela (Bolivarian Republic of)": "Venezuela",
    "Venezuela, Bolivarian Republic of": "Venezuela",
    "Venezuela, RB": "Venezuela",
    "Venezuela, Rep. Bol.": "Venezuela",
    "Virgin Islands (U.S.)": "United States Virgin Islands",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "Slovak Republic": "Slovakia",
    "Czechia": "Czech Republic",
    "Kyrgyzstan": "Kyrgyz Republic",
    "Lao PDR": "Lao People's Democratic Republic",
    "Yemen, Rep.": "Yemen",
    "Syrian Arab Republic": "Syria",
    "Bosnia & Herzegovina": "Bosnia and Herzegovina",
    "Libyan Arab Jamahiriya": "Libya"
}

for df in all_data_frames:
    old_shape = df.shape
    df["Country"].replace(replace_names_dict, inplace = True)
    assert df.shape == old_shape

In [30]:
final_df = reduce(lambda x, y: pd.merge(x, y, on = "Country", how = "outer", sort = "True"), all_data_frames)
# a manual fix for Congo data
final_df.at[86, "Early education attendance rate"] = final_df.at[99, "Early education attendance rate"]
# Drop all rows with more than 70% NaN's
final_df.dropna(thresh = int((len(final_df.columns) - 1) * 0.7), inplace = True)
final_df.to_csv("data/synthesized_data.csv", index = False)
# display(final_df.Country.unique())
# final_df[final_df.Country.str.contains("Korea")]
final_df

,Country,Population (thousands),Compulsory attendence amount,Early education attendance rate,Primary enrollment,Secondary enrollment,Tertiary enrollment,Male literacy rate,Female literacy rate,Primary completion rate,...,Upper secondary teacher-student ratio,Education spending (percentage of GDP),Unemployment rate,Child labor percentage,Average income,Average income currency,Average Income GNI,GDP (USD),Labor force count,Three year average unnourished count
1,Afghanistan,29825,9.0,1.0,4318819,651453,27648,43.1,12.6,84.330589,...,30.74908,3.21378,11.73,10.0,NaN,NaN,500.0,1.980707e+10,NaN,11.1
9,Albania,3162,9.0,40.0,250487,397056,53014,99.2,98.3,100.732407,...,17.65478,3.94576,11.70,12.0,NaN,NaN,5210.0,1.479962e+10,1102890.0,0.1
10,Algeria,38482,10.0,NaN,4361744,3755821,755463,79.6,60.1,103.663528,...,16.15822,6.10036,12.83,5.0,NaN,NaN,3550.0,1.451639e+11,10500000.0,1.2
18,Angola,20821,6.0,NaN,1057188,413695,12566,82.9,54.2,46.187149,...,20.50581,1.82118,7.70,24.0,NaN,NaN,2230.0,6.230691e+10,NaN,5.7
20,Antigua and Barbuda,89,11.0,NaN,13025,5276,NaN,NaN,NaN,96.198830,...,13.17411,3.32162,NaN,NaN,NaN,NaN,14250.0,1.415059e+09,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,Venezuela,29955,17.0,NaN,3449290,2028388,983217,93.3,92.7,92.588737,...,NaN,1.34408,9.14,8.0,106141.0,Bolívar soberano,13080.0,4.823593e+11,12379700.0,9.1
401,Vietnam,90796,10.0,72.0,7773484,9939319,1354543,93.9,86.9,110.021210,...,27.60630,4.06197,2.27,7.0,NaN,NaN,2660.0,2.711584e+11,NaN,6.1
415,Yemen,23852,9.0,3.0,3219554,1455216,201043,73.1,34.7,72.300392,...,24.17273,5.15143,13.42,23.0,NaN,NaN,940.0,2.348627e+10,NaN,NaN
416,Zambia,14075,7.0,NaN,2565419,408971,24553,76.3,59.8,79.973877,...,NaN,4.46518,12.17,41.0,NaN,NaN,1190.0,1.932005e+10,NaN,NaN
